<a href="https://colab.research.google.com/github/larissabooth/cv_project/blob/main/evaluate_ss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: ignored

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.


from pathlib import Path
import argparse
import json
import os
import random
import signal
import sys
import time
import urllib

from torch import nn, optim
from torchvision import datasets, transforms
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

# from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - more specific here
# writer = SummaryWriter('runs/FT_exp_3')

sys.path.append("/content/drive/My Drive/computer_vision_project/vicreg")

import resnet

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#@title Network configurations

%cd "/content/drive/My Drive/computer_vision_project/vicreg"

from datetime import datetime
currentDateAndTime = datetime.now()

currentTime = currentDateAndTime.strftime("%H_%M")

#Data
data_dir = Path("/content/drive/My Drive/computer_vision_project/Kitchener_torch")
train_percent = .1 #"size of traing set in percent"
test_percent = 1.0

# Checkpoint
ckpt_file = Path("checkpoint_"+currentTime+".pth")
pretrained = "./checkpoints/self_sup/500_epochs/resnet50_bb.pth" #path to pretrained model
exp_dir = Path("./checkpoints/ss_lincls") #path to export directory
print_freq = 5 #number of steps before printing

# Model
arch = "resnet50"

# Optim
epochs = 100
batch_size = 64
lr_backbone = 0.0 #"backbone base learning rate"
lr_head = 0.01 #"classifier base learning rate"
weight_decay = 0 #1e-6
weights = "freeze" #("finetune", "freeze")

# Running
workers= 8 #"number of data loader workers"
rank = 0

#Stats file
train_stats_file =open("./stats/ss_lincls/train_stats_lr_"+str(lr_head)+"_"+currentTime+".json", "a", buffering=1)
val_stats_file =open("./stats/ss_lincls/val_stats_lr_"+str(lr_head)+"_"+currentTime+".json", "a", buffering=1)
test_stats_file =open("./stats/ss_lincls/test_stats_lr_"+str(lr_head)+"_"+currentTime+".json", "a", buffering=1)

[Errno 2] No such file or directory: '/content/drive/My Drive/computer_vision_project/vicreg'
/home/lara


NameError: ignored

In [ ]:
#@title Accuracy helpers

import argparse

def calc_accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=":f"):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)

In [ ]:
#@title Class names
classes = (
"acer",
"tilia",
"fraxinus", 
"ulmus", 
"sorbus", 
"amelanchier", 
"crataegus", 
"gleditsia", 
"prunus", 
"quercus", 
"picea", 
"populus", 
"platanus", 
"pyrus", 
"salix", 
"fagus", 
"gymnocladus", 
"cornus", 
"betula", 
"taxus", 
"ginkgo", 
"magnolia", 
"pinus", 
"aesculus", 
"abies", 
"corylus")

In [ ]:
#@title Load model

backbone, embedding = resnet.__dict__[arch](zero_init_residual=True)
state_dict = torch.load(pretrained, map_location="cpu")
if "model" in state_dict:
    state_dict = state_dict["model"]
    state_dict = {
        key.replace("backbone.", ""): value
        for (key, value) in state_dict.items()
    }
backbone.load_state_dict(state_dict, strict=False)

head = nn.Linear(embedding, 26)
head.weight.data.normal_(mean=0.0, std=0.01)
head.bias.data.zero_()
model = nn.Sequential(backbone, head)
model.to(device)

if weights == "freeze":
    backbone.requires_grad_(False)
    head.requires_grad_(True)

criterion = nn.CrossEntropyLoss().to(device)

param_groups = [dict(params=head.parameters(), lr=lr_head)]
if weights == "finetune":
    param_groups.append(dict(params=backbone.parameters(), lr=lr_backbone))
optimizer = optim.SGD(param_groups, 0, momentum=0.9, weight_decay=weight_decay)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

# automatically resume from checkpoint if it exists
if (exp_dir / ckpt_file).is_file():
    ckpt = torch.load(exp_dir / ckpt_file, map_location="cpu")
    start_epoch = ckpt["epoch"]
    best_acc = ckpt["best_acc"]
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    # scheduler.load_state_dict(ckpt["scheduler"])
else:
    start_epoch = 0
    best_acc = argparse.Namespace(top1=0, top5=0)

In [ ]:
#@title Data loading code
from torch.utils.data import random_split
traindir = data_dir / "train"
valdir = data_dir / "val"
testdir = data_dir / "test"

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

#training data
train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose(
        [
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]
    ),
)

actual_train_dataset, val_dataset, extra = random_split(train_dataset, [train_percent, train_percent, 1.0-train_percent*2])



# testing data
test_dataset = datasets.ImageFolder(
    testdir,
    transforms.Compose(
        [
            # transforms.Resize(256),
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ]
    ),
)

actual_test_dataset, spare = random_split(test_dataset, [test_percent, 1.0 - test_percent])

kwargs = dict(
    batch_size=batch_size,
    num_workers=workers,
    shuffle = True,
    pin_memory = True
)
train_loader = torch.utils.data.DataLoader(actual_train_dataset,  **kwargs)
val_loader = torch.utils.data.DataLoader(val_dataset, **kwargs)
test_loader = torch.utils.data.DataLoader(actual_test_dataset, **kwargs)

In [ ]:
#@title Count the number of images per label

from collections import Counter

label_counts = Counter(train_dataset.targets)

# Find the frequent, common, and rare labels
freq_labels = []
common_labels = []
rare_labels = []

for label, count in label_counts.items():
    if count > 1000:
        freq_labels.append(label)
    elif count > 100:
        common_labels.append(label)
    else:
        rare_labels.append(label)

# Print the results
print("Frequent labels:", freq_labels)
print("Common labels:", common_labels)
print("Rare labels:", rare_labels)

In [ ]:
# import numpy as np
# subset_classes = [0, 1, 2]
# preds = np.array([0, 1, 1, 0, 2, 1, 2, 0, 0, 1, 3, 4])
# labels =np.array([0, 1, 2, 0, 2, 1, 2, 0, 0, 1, 5, 5])
# num_classes = 6
def get_class_sums(preds, labels, num_classes):
  numer = np.zeros(num_classes)
  denom = np.zeros(num_classes)
  correct = (preds == labels)
  pred_unique , pred_counts = np.unique(labels[correct], return_counts=True)
  unique , counts = np.unique(labels, return_counts=True)
  numer[pred_unique] = pred_counts
  denom[unique] = counts
  return numer, denom
# numer, denom = get_class_sums(preds, labels, num_classes)
def get_class_ave_recall(classes, numer, denom):
  class_numer = numer[classes]
  class_denom = denom[classes]
  nonzero_denom = np.nonzero(class_denom)
  print("class_numer", class_numer, "class_denom", class_denom, "nonzero", nonzero_denom)
  if nonzero_denom[0].size == 0:
    return -1.0
  class_numer = class_numer[nonzero_denom]
  class_denom = class_denom[nonzero_denom]
  summed = np.sum(np.divide(class_numer, class_denom))
  return summed/class_denom.size
# print(get_class_ave_recall(subset_classes, numer, denom)) # answer should be 0.88888

In [ ]:
#@title training code
NUM_CLASSES = 26
def train():
    start_time = time.time()

    #total
    correct = 0.0
    total = 0.0

    for epoch in range(start_epoch, epochs):
        # train
        if weights == "finetune":
            model.train()
        elif weights == "freeze":
            model.eval()
        else:
            assert False
        print("Starting epoch:", epoch)
        for step, (images, target) in enumerate(
            train_loader, start=epoch * len(train_loader)
        ):
            output = model(images.to(device))
            _, preds = torch.max(output, 1)

            loss = criterion(output, target.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #ground truth and predictions
            labels = target.cpu().numpy()
            preds = preds.cpu().numpy()

            #total
            correct += np.sum(preds == labels)
            total += labels.size
            
            #compute recall & accuracy
            
            if step % print_freq == 0:
                pg = optimizer.param_groups
                lr_head = pg[0]["lr"]
                lr_backbone = pg[1]["lr"] if len(pg) == 2 else 0

                accuracy = correct / float(total)
                #record current stats
                stats = dict(
                    epoch=epoch,
                    step=step,
                    lr_backbone=lr_backbone,
                    lr_head=lr_head,
                    loss=loss.item,
                    accuracy=accuracy*100,
                    time=int(time.time() - start_time),
                )

                #total
                correct = 0.0
                total = 0.0

                # print(json.dumps(stats))
                print(json.dumps(stats), file=train_stats_file)
        print("finished training epoch")


        # evaluate
        model.eval()
        if rank == 0:
            top1 = AverageMeter("Acc@1")
            top5 = AverageMeter("Acc@5")
            with torch.no_grad():
                sum_numer = np.zeros(NUM_CLASSES)
                sum_denom = np.zeros(NUM_CLASSES)

                for images, target in val_loader:
                    output = model(images.to(device))
                    acc1, acc5 = calc_accuracy(
                        output, target.to(device), topk=(1, 5)
                    )
                    top1.update(acc1[0].item(), images.size(0))
                    top5.update(acc5[0].item(), images.size(0))
                    _, preds = torch.max(output, 1)
                    preds = preds.cpu().numpy()

                    labels = target.cpu().numpy()
                    loss = criterion(output, target.to(device))

                    numer, denom = get_class_sums(preds, labels, NUM_CLASSES)

                    sum_numer += numer
                    sum_denom += denom

            best_acc.top1 = max(best_acc.top1, top1.avg)
            best_acc.top5 = max(best_acc.top5, top5.avg)
            
            #compute recalls
            freq_recall = get_class_ave_recall(freq_labels, sum_numer, sum_denom)
            common_recall = get_class_ave_recall(common_labels, sum_numer, sum_denom)
            rare_recall = get_class_ave_recall(rare_labels, sum_numer, sum_denom)
            class_avg_recall = get_class_ave_recall([i for i in range(NUM_CLASSES)], sum_numer, sum_denom)

            stats = dict(
                epoch=epoch,
                acc1=top1.avg,
                acc5=top5.avg,
                loss=loss.item(),
                freq_recall=freq_recall,
                common_recall=common_recall,
                rare_recall=rare_recall,
                class_avg_recall=class_avg_recall,
                best_acc1=best_acc.top1,
                best_acc5=best_acc.top5,
            )
            # print("val_stats", stats)
            print(json.dumps(stats))
            print(json.dumps(stats), file=val_stats_file)
        # scheduler.step()
        if rank == 0:
            state = dict(
                epoch=epoch + 1,
                best_acc=best_acc,
                model=model.state_dict(),
                optimizer=optimizer.state_dict(),
                # scheduler=scheduler.state_dict(),
            )
            torch.save(state, exp_dir / ckpt_file)
        print("finishing STEP", step)

# Copyright (c) Meta Platforms, Inc. and affiliates.

# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

In [ ]:
#@title testing code 

def test():
        # evaluate
        model.eval()
        if rank == 0:
            top1_test = AverageMeter("Acc@1")
            top5_test = AverageMeter("Acc@5")
            with torch.no_grad():
                sum_numer = np.zeros(NUM_CLASSES)
                sum_denom = np.zeros(NUM_CLASSES)
                for images, target in test_loader:
                    output = model(images.to(device))
                    acc1, acc5 = calc_accuracy(
                        output, target.to(device), topk=(1, 5)
                    )
                    top1_test.update(acc1[0].item(), images.size(0))
                    top5_test.update(acc5[0].item(), images.size(0))
                    _, preds = torch.max(output, 1)
                    preds = preds.cpu().numpy()

                    labels = target.cpu().numpy()

                    numer, denom = get_class_sums(preds, labels, NUM_CLASSES)

                    sum_numer += numer
                    sum_denom += denom
            
            #compute recalls
            freq_recall = get_class_ave_recall(freq_labels, sum_numer, sum_denom)
            common_recall = get_class_ave_recall(common_labels, sum_numer, sum_denom)
            rare_recall = get_class_ave_recall(rare_labels, sum_numer, sum_denom)
            class_avg_recall = get_class_ave_recall([i for i in range(NUM_CLASSES)], sum_numer, sum_denom)

            stats = dict(
                acc1=top1_test.avg,
                acc5=top5_test.avg,
                freq_recall=freq_recall,
                common_recall=common_recall,
                rare_recall=rare_recall,
                class_avg_recall=class_avg_recall
            )
            # print("test_stats", stats)
            print(json.dumps(stats))
            print(json.dumps(stats), file=test_stats_file)

In [ ]:
train()

In [ ]:
test()